In [2]:
import pandas as pd, numpy as np

In [4]:
df = pd.read_csv(r"C:\Users\Owner\OneDrive\NursingHomeData\pbj-nurse\pbj_nurse_2017_Q1.csv")

C:\Users\Owner\AppData\Local\Temp\ipykernel_24916\4143783690.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\Owner\OneDrive\NursingHomeData\pbj-nurse\pbj_nurse_2017_Q1.csv")


In [11]:
print(df.head())

for col in df.columns:
    print(col)

print(df.info())

  provnum                  provname          city state county_name  \
0   15009  BURNS NURSING HOME, INC.  RUSSELLVILLE    AL    Franklin   
1   15009  BURNS NURSING HOME, INC.  RUSSELLVILLE    AL    Franklin   
2   15009  BURNS NURSING HOME, INC.  RUSSELLVILLE    AL    Franklin   
3   15009  BURNS NURSING HOME, INC.  RUSSELLVILLE    AL    Franklin   
4   15009  BURNS NURSING HOME, INC.  RUSSELLVILLE    AL    Franklin   

   county_fips  cy_qtr  workdate  mdscensus  hrs_rn_donadmin  ...  \
0           59  2017Q1  20170101         55              0.0  ...   
1           59  2017Q1  20170102         54              0.0  ...   
2           59  2017Q1  20170103         55              0.0  ...   
3           59  2017Q1  20170104         55              0.0  ...   
4           59  2017Q1  20170105         56              0.0  ...   

   hrs_lpn_ctr  hrs_cna  hrs_cna_emp  hrs_cna_ctr  hrs_na_trn  hrs_natrn_emp  \
0          0.0   121.00       121.00          0.0         0.0            0.0  

In [18]:
# PBJ nurse cleaner — standardize names & dtypes (reads from pbj-nurse/, no saving)

from __future__ import annotations
import os, re, warnings
from pathlib import Path
from typing import Dict, Tuple, List
import pandas as pd
import numpy as np

# ── Paths ────────────────────────────────────────────────────────────────────
PROJECT_ROOT = Path.cwd()
while not (PROJECT_ROOT / "src").is_dir() and PROJECT_ROOT != PROJECT_ROOT.parent:
    PROJECT_ROOT = PROJECT_ROOT.parent

RAW_DIR  = Path(os.getenv("NH_DATA_DIR", PROJECT_ROOT / "data" / "raw")).resolve()
PBJ_DIR  = RAW_DIR / "pbj-nurse"   # <<— your folder
PBJ_GLOB = "pbj_nurse_????_Q[1-4].csv"
FILENAME_RE = re.compile(r"pbj_nurse_(\d{4})_Q([1-4])\.csv$", re.IGNORECASE)

print(f"[paths] RAW_DIR={RAW_DIR}")
print(f"[paths] PBJ_DIR={PBJ_DIR}")

# ── Config: renames / schema ─────────────────────────────────────────────────
RENAME_MAP = {
    "provnum":          "cms_certification_number",
    "provname":         "provider_name",
    "city":             "city",
    "state":            "state",
    "county_name":      "county_name",
    "county_fips":      "county_fips",
    "cy_qtr":           "cy_qtr",
    "workdate":         "workdate",
    "mdscensus":        "mds_census",
    "hrs_rn_donadmin":  "hrs_rn_don_admin",
    "hrs_rndon_emp":    "hrs_rn_don_emp",
    "hrs_rndon_ctr":    "hrs_rn_don_ctr",
    "hrs_rnadmin":      "hrs_rn_admin",
    "hrs_rnadmin_emp":  "hrs_rn_admin_emp",
    "hrs_rnadmin_ctr":  "hrs_rn_admin_ctr",
    "hrs_rn":           "hrs_rn",
    "hrs_rn_emp":       "hrs_rn_emp",
    "hrs_rn_ctr":       "hrs_rn_ctr",
    "hrs_lpn_admin":    "hrs_lpn_admin",
    "hrs_lpnadmin_emp": "hrs_lpn_admin_emp",
    "hrs_lpnadmin_ctr": "hrs_lpn_admin_ctr",
    "hrs_lpn":          "hrs_lpn",
    "hrs_lpn_emp":      "hrs_lpn_emp",
    "hrs_lpn_ctr":      "hrs_lpn_ctr",
    "hrs_cna":          "hrs_cna",
    "hrs_cna_emp":      "hrs_cna_emp",
    "hrs_cna_ctr":      "hrs_cna_ctr",
    "hrs_na_trn":       "hrs_na_trn",
    "hrs_natrn_emp":    "hrs_na_trn_emp",
    "hrs_natrn_ctr":    "hrs_na_trn_ctr",
    "hrs_medaide":      "hrs_medaide",
    "hrs_medaide_emp":  "hrs_medaide_emp",
    "hrs_medaide_ctr":  "hrs_medaide_ctr",
}

HOUR_COLS = [
    "hrs_rn_don_admin","hrs_rn_don_emp","hrs_rn_don_ctr",
    "hrs_rn_admin","hrs_rn_admin_emp","hrs_rn_admin_ctr",
    "hrs_rn","hrs_rn_emp","hrs_rn_ctr",
    "hrs_lpn_admin","hrs_lpn_admin_emp","hrs_lpn_admin_ctr",
    "hrs_lpn","hrs_lpn_emp","hrs_lpn_ctr",
    "hrs_cna","hrs_cna_emp","hrs_cna_ctr",
    "hrs_na_trn","hrs_na_trn_emp","hrs_na_trn_ctr",
    "hrs_medaide","hrs_medaide_emp","hrs_medaide_ctr",
]

CANON_ORDER = [
    "cms_certification_number","provider_name","city","state","county_name","county_fips",
    "cy_qtr","year","quarter","workdate","mds_census",
    *HOUR_COLS,
    "source_file"
]

# ── Helpers ──────────────────────────────────────────────────────────────────
def _parse_filename_quarter(fp: Path) -> Tuple[int, int]:
    m = FILENAME_RE.search(fp.name)
    if not m:
        raise ValueError(f"Filename does not match pattern YYYY_Qn: {fp.name}")
    return int(m.group(1)), int(m.group(2))

def _to_datetime_from_yyyymmdd_int(s: pd.Series) -> pd.Series:
    return pd.to_datetime(s.astype("Int64"), format="%Y%m%d", errors="coerce")

def _zp6(s: pd.Series) -> pd.Series:
    s = s.astype("string").str.strip()
    s = s.where(s.ne("<NA>"))
    return s.str.zfill(6)

def _clean_one(df_raw: pd.DataFrame, src: Path) -> pd.DataFrame:
    df = df_raw.copy()
    df.columns = [c.strip().lower() for c in df.columns]
    for old, new in RENAME_MAP.items():
        if old in df.columns:
            df.rename(columns={old: new}, inplace=True)

    # Ensure hour columns exist
    for c in HOUR_COLS:
        if c not in df.columns:
            df[c] = 0.0

    # ID key
    if "cms_certification_number" not in df.columns and "provnum" in df.columns:
        df.rename(columns={"provnum":"cms_certification_number"}, inplace=True)
    if "cms_certification_number" in df.columns:
        df["cms_certification_number"] = _zp6(df["cms_certification_number"])
    else:
        warnings.warn(f"{src.name}: missing cms_certification_number/provnum")

    # Strings & casing
    for col in ["provider_name","city","state","county_name","cy_qtr"]:
        if col in df.columns:
            df[col] = df[col].astype("string").str.strip()
    if "state" in df.columns: df["state"] = df["state"].str.upper()
    if "city" in df.columns: df["city"] = df["city"].str.title()
    if "county_name" in df.columns: df["county_name"] = df["county_name"].str.title()

    # county_fips
    if "county_fips" in df.columns:
        df["county_fips"] = pd.to_numeric(df["county_fips"], errors="coerce").astype("Int64")

    # workdate
    if "workdate" in df.columns:
        if pd.api.types.is_integer_dtype(df["workdate"]) or pd.api.types.is_string_dtype(df["workdate"]):
            df["workdate"] = _to_datetime_from_yyyymmdd_int(df["workdate"])
        else:
            df["workdate"] = pd.to_datetime(df["workdate"], errors="coerce")

    # mds_census
    for c in ["mds_census","mdscensus"]:
        if c in df.columns:
            df["mds_census"] = pd.to_numeric(df[c], errors="coerce").astype("Int32")
            if c != "mds_census":
                df.drop(columns=[c], inplace=True)

    # hours -> float32
    for c in HOUR_COLS:
        df[c] = pd.to_numeric(df[c], errors="coerce").astype("float32").fillna(0.0)

    # year/quarter from filename; check cy_qtr
    file_year, file_q = _parse_filename_quarter(src)
    df["year"] = file_year
    df["quarter"] = file_q

    if "cy_qtr" in df.columns and df["cy_qtr"].notna().any():
        df["cy_qtr"] = df["cy_qtr"].str.upper().str.replace(r"\s+", "", regex=True)
        sample = df["cy_qtr"].dropna().astype(str).head(1)
        if not sample.empty:
            expected = f"{file_year}Q{file_q}"
            if sample.iloc[0] != expected:
                warnings.warn(f"{src.name}: filename quarter={expected} != cy_qtr sample={sample.iloc[0]}")
    else:
        df["cy_qtr"] = pd.Series([f"{file_year}Q{file_q}"]*len(df), dtype="string")

    df["source_file"] = src.name

    # Reorder; keep extras at end
    for col in CANON_ORDER:
        if col not in df.columns:
            df[col] = pd.NA
    extras = [c for c in df.columns if c not in CANON_ORDER]
    df = df[CANON_ORDER + extras]
    return df

# --- Add these helpers above load_and_clean_pbj ---
import pandas as pd

_TRY_ENCODINGS = ["utf-8", "utf-8-sig", "cp1252", "latin1"]

def _read_csv_robust(fp: Path) -> pd.DataFrame:
    """
    Try a few common encodings; fall back to cp1252/latin1 if needed.
    Also tolerates a few bad lines without crashing.
    """
    last_err = None
    for enc in _TRY_ENCODINGS:
        try:
            df = pd.read_csv(
                fp,
                low_memory=False,
                encoding=enc,
                encoding_errors="strict",   # raise first so we can try the next encoding
                on_bad_lines="error",       # raise first so we can try fallback below
            )
            # If that worked, we're done.
            print(f"[read] {fp.name}  (encoding={enc})")
            return df
        except Exception as e:
            last_err = e

    # Final fallback: allow replacement + skip bad lines if any remain problematic
    for enc in ["cp1252", "latin1"]:
        try:
            df = pd.read_csv(
                fp,
                low_memory=False,
                encoding=enc,
                encoding_errors="replace",  # replace undecodable bytes
                on_bad_lines="skip",        # skip malformed rows (rare)
            )
            print(f"[read] {fp.name}  (encoding={enc}, replace+skip)")
            return df
        except Exception as e:
            last_err = e

    raise last_err

def load_and_clean_pbj(return_concat: bool = True):
    files = sorted(PBJ_DIR.glob(PBJ_GLOB))
    if not files:
        raise FileNotFoundError(f"No files matched {PBJ_DIR / PBJ_GLOB}")
    print(f"[scan] found {len(files)} files under {PBJ_DIR}")
    for f in files[:5]:
        print("  •", f.name)

    cleaned: Dict[str, pd.DataFrame] = {}
    for fp in files:
        df_raw = _read_csv_robust(fp)          # <-- use robust reader
        df = _clean_one(df_raw, fp)
        cleaned[fp.name] = df

    panel = pd.concat(cleaned.values(), axis=0, ignore_index=True) if return_concat else None
    return cleaned, panel

# ---- Example usage (uncomment to run) ----
cleaned_map, panel_df = load_and_clean_pbj(return_concat=True)
# • cleaned_map: dict mapping {filename -> cleaned DataFrame}
# • panel_df   : single long DataFrame (not saved)
panel_df.head()

[paths] RAW_DIR=C:\Users\Owner\OneDrive\NursingHomeData
[paths] PBJ_DIR=C:\Users\Owner\OneDrive\NursingHomeData\pbj-nurse
[scan] found 30 files under C:\Users\Owner\OneDrive\NursingHomeData\pbj-nurse
  • pbj_nurse_2017_Q1.csv
  • pbj_nurse_2017_Q2.csv
  • pbj_nurse_2017_Q3.csv
  • pbj_nurse_2017_Q4.csv
  • pbj_nurse_2018_Q1.csv
[read] pbj_nurse_2017_Q1.csv  (encoding=utf-8)
[read] pbj_nurse_2017_Q2.csv  (encoding=utf-8)
[read] pbj_nurse_2017_Q3.csv  (encoding=utf-8)
[read] pbj_nurse_2017_Q4.csv  (encoding=utf-8)
[read] pbj_nurse_2018_Q1.csv  (encoding=cp1252)
[read] pbj_nurse_2018_Q2.csv  (encoding=cp1252)
[read] pbj_nurse_2018_Q3.csv  (encoding=cp1252)
[read] pbj_nurse_2018_Q4.csv  (encoding=utf-8)
[read] pbj_nurse_2019_Q1.csv  (encoding=utf-8)
[read] pbj_nurse_2019_Q2.csv  (encoding=utf-8)
[read] pbj_nurse_2019_Q3.csv  (encoding=utf-8)
[read] pbj_nurse_2019_Q4.csv  (encoding=utf-8)
[read] pbj_nurse_2020_Q1.csv  (encoding=cp1252)
[read] pbj_nurse_2021_Q1.csv  (encoding=cp1252)
[read] 

,cms_certification_number,provider_name,city,state,county_name,county_fips,cy_qtr,year,quarter,workdate,...,hrs_na_trn_emp,hrs_na_trn_ctr,hrs_medaide,hrs_medaide_emp,hrs_medaide_ctr,source_file,hrs_rndon,hrs_lpnadmin,hrs_natrn,incomplete
0,015009,"BURNS NURSING HOME, INC.",Russellville,AL,Franklin,59,2017Q1,2017,1,2017-01-01,...,0.0,0.0,0.0,0.0,0.0,pbj_nurse_2017_Q1.csv,NaN,NaN,NaN,NaN
1,015009,"BURNS NURSING HOME, INC.",Russellville,AL,Franklin,59,2017Q1,2017,1,2017-01-02,...,0.0,0.0,0.0,0.0,0.0,pbj_nurse_2017_Q1.csv,NaN,NaN,NaN,NaN
2,015009,"BURNS NURSING HOME, INC.",Russellville,AL,Franklin,59,2017Q1,2017,1,2017-01-03,...,0.0,0.0,0.0,0.0,0.0,pbj_nurse_2017_Q1.csv,NaN,NaN,NaN,NaN
3,015009,"BURNS NURSING HOME, INC.",Russellville,AL,Franklin,59,2017Q1,2017,1,2017-01-04,...,0.0,0.0,0.0,0.0,0.0,pbj_nurse_2017_Q1.csv,NaN,NaN,NaN,NaN
4,015009,"BURNS NURSING HOME, INC.",Russellville,AL,Franklin,59,2017Q1,2017,1,2017-01-05,...,0.0,0.0,0.0,0.0,0.0,pbj_nurse_2017_Q1.csv,NaN,NaN,NaN,NaN


In [24]:
panel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39867242 entries, 0 to 39867241
Data columns (total 40 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   cms_certification_number  string        
 1   provider_name             string        
 2   city                      string        
 3   state                     string        
 4   county_name               string        
 5   county_fips               Int64         
 6   cy_qtr                    string        
 7   year                      int64         
 8   quarter                   int64         
 9   workdate                  datetime64[ns]
 10  mds_census                Int32         
 11  hrs_rn_don_admin          float32       
 12  hrs_rn_don_emp            float32       
 13  hrs_rn_don_ctr            float32       
 14  hrs_rn_admin              float32       
 15  hrs_rn_admin_emp          float32       
 16  hrs_rn_admin_ctr          float32       
 17  hrs_rn

In [23]:
print(panel_df.describe())

       county_fips          year       quarter                       workdate  \
count   39867150.0  3.986724e+07  3.986724e+07                       39867242   
mean     90.933138  2.020709e+03  2.417612e+00  2021-03-09 05:26:12.494692864   
min            1.0  2.017000e+03  1.000000e+00            2017-01-01 00:00:00   
25%           31.0  2.018000e+03  1.000000e+00            2018-11-22 00:00:00   
50%           69.0  2.021000e+03  2.000000e+00            2021-07-08 00:00:00   
75%          117.0  2.023000e+03  3.000000e+00            2023-05-18 00:00:00   
max          840.0  2.025000e+03  4.000000e+00            2025-03-31 00:00:00   
std      98.886944  2.486311e+00  1.140505e+00                            NaN   

       mds_census  hrs_rn_don_admin  hrs_rn_don_emp  hrs_rn_don_ctr  \
count  39867242.0      3.986724e+07    3.986724e+07    3.986724e+07   
mean    82.932671      2.454517e-01    4.969958e+00    8.782741e-02   
min           0.0      0.000000e+00    0.000000e+00    0.

In [29]:
print(panel_df.isnull().sum())

cms_certification_number           0
provider_name                    184
city                               0
state                              0
county_name                        0
county_fips                       92
cy_qtr                             0
year                               0
quarter                            0
workdate                           0
mds_census                         0
hrs_rn_don_admin                   0
hrs_rn_don_emp                     0
hrs_rn_don_ctr                     0
hrs_rn_admin                       0
hrs_rn_admin_emp                   0
hrs_rn_admin_ctr                   0
hrs_rn                             0
hrs_rn_emp                         0
hrs_rn_ctr                         0
hrs_lpn_admin                      0
hrs_lpn_admin_emp                  0
hrs_lpn_admin_ctr                  0
hrs_lpn                            0
hrs_lpn_emp                        0
hrs_lpn_ctr                        0
hrs_cna                            0
h

In [30]:
# 1. Aggregate daily RN, LPN, NA
agg_daily = (
    panel_df
    .groupby(["cms_certification_number", "workdate"], as_index=False)
    .agg({
        "hrs_rn": "sum",
        "hrs_lpn": "sum",
        "hrs_cna": "sum",
    })
)

# 2. Total hours
agg_daily["total_hours"] = agg_daily[["hrs_rn", "hrs_lpn", "hrs_cna"]].sum(axis=1)

# 3. Add year-month column for monthly checks
agg_daily["year_month"] = agg_daily["workdate"].dt.to_period("M")

# 4. Reporting coverage per facility-month
days_in_month = agg_daily["workdate"].dt.days_in_month
agg_daily = agg_daily.assign(days_in_month=days_in_month)

coverage = (
    agg_daily.groupby(["cms_certification_number", "year_month"])
    .agg(days_reported=("workdate", "nunique"),
         days_in_month=("days_in_month", "max"))
    .reset_index()
)

coverage["coverage_ratio"] = coverage["days_reported"] / coverage["days_in_month"]
underreported = coverage[coverage["coverage_ratio"] < 0.5]

print(f"[coverage] Found {len(underreported)} facility-months with <50% daily reporting")

# 5. Outlier detection per facility-month
def flag_outliers(x, cols=("hrs_rn","hrs_lpn","hrs_cna","total_hours"), thresh=3.0):
    """
    Flag daily rows where any hours column is more than thresh*std from mean
    within facility-month.
    """
    outlier_mask = pd.Series(False, index=x.index)
    for col in cols:
        mu = x[col].mean()
        sigma = x[col].std()
        if sigma > 0:
            outlier_mask |= (np.abs(x[col] - mu) > thresh * sigma)
    x = x.assign(outlier=outlier_mask)
    return x

agg_daily = (
    agg_daily
    .groupby(["cms_certification_number", "year_month"], group_keys=False)
    .apply(flag_outliers)
)

print(f"[outliers] {agg_daily['outlier'].sum():,} daily records flagged as outliers")

# 6. Remove underreported months + outliers
valid_months = coverage.loc[coverage["coverage_ratio"] >= 0.5,
                            ["cms_certification_number","year_month"]]
agg_clean = (
    agg_daily
    .merge(valid_months, on=["cms_certification_number","year_month"])
    .loc[lambda d: ~d["outlier"]]
    .drop(columns=["outlier","days_in_month"])
    .reset_index(drop=True)
)

print(f"[final] rows before={len(agg_daily):,}, after cleaning={len(agg_clean):,}")

agg_clean.head()

[coverage] Found 725 facility-months with <50% daily reporting


MemoryError: Unable to allocate 152. MiB for an array with shape (1, 39867242) and data type int32